# Analisis de Vegetacion (NDVI) en San Juan de Pasto

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/GeoIA-Pasto/notebooks/03_Analisis_NDVI_Vegetacion_Pasto.ipynb)

---

## Objetivos

1. Calcular el Indice de Vegetacion de Diferencia Normalizada (NDVI)
2. Analizar la distribucion de vegetacion en Pasto
3. Comparar vegetacion urbana vs areas naturales
4. Monitorear el Volcan Galeras y la Laguna de la Cocha

## 1. Instalacion de Dependencias

In [ ]:
%pip install -q geoai-py leafmap pystac-client planetary-computer rioxarray matplotlib

## 2. Importacion de Librerias

In [ ]:
import geoai
import leafmap
import rioxarray as rxr
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pystac_client
import planetary_computer
import os
import warnings

warnings.filterwarnings("ignore")

print(f"GeoAI version: {geoai.__version__}")

## 3. Definicion del Area de Estudio

In [ ]:
# Coordenadas de San Juan de Pasto
PASTO_CENTER = {"lat": 1.2136, "lon": -77.2811}

# Bounding Box que incluye el area urbana y alrededores
PASTO_BBOX = (-77.35, 1.15, -77.20, 1.30)

# Areas de interes especificas
AREAS = {
    "Centro Urbano": (-77.30, 1.20, -77.26, 1.24),
    "Volcan Galeras": (-77.40, 1.18, -77.32, 1.26),
    "Zona Rural Norte": (-77.30, 1.25, -77.22, 1.30),
}

print(f"Area de estudio: {PASTO_BBOX}")

## 4. Buscar y Descargar Imagen Sentinel-2

In [ ]:
# Conectar a Planetary Computer
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

# Buscar imagenes con pocas nubes
search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=PASTO_BBOX,
    datetime="2024-01-01/2024-12-31",
    query={"eo:cloud_cover": {"lt": 15}},
)

items = list(search.items())
print(f"Imagenes encontradas: {len(items)}")

# Seleccionar la mejor imagen
if items:
    best_item = min(items, key=lambda x: x.properties.get("eo:cloud_cover", 100))
    print(f"\nImagen seleccionada: {best_item.id}")
    print(f"Fecha: {best_item.datetime}")
    print(f"Nubes: {best_item.properties.get('eo:cloud_cover')}%")

In [ ]:
# Cargar bandas necesarias para NDVI
# NDVI = (NIR - RED) / (NIR + RED)
# Sentinel-2: NIR = B08, RED = B04

if items:
    red_url = best_item.assets["B04"].href  # Banda Roja
    nir_url = best_item.assets["B08"].href  # Banda NIR

    print("Cargando bandas...")

    # Cargar y recortar al area de estudio
    red = rxr.open_rasterio(red_url).squeeze()
    red = red.rio.clip_box(*PASTO_BBOX)

    nir = rxr.open_rasterio(nir_url).squeeze()
    nir = nir.rio.clip_box(*PASTO_BBOX)

    print(f"Banda Roja shape: {red.shape}")
    print(f"Banda NIR shape: {nir.shape}")

## 5. Calcular NDVI

El **Indice de Vegetacion de Diferencia Normalizada (NDVI)** es un indicador que permite evaluar la presencia y salud de la vegetacion:

$$NDVI = \frac{NIR - RED}{NIR + RED}$$

**Interpretacion:**
- **-1 a 0**: Agua, nubes, nieve, superficies artificiales
- **0 a 0.2**: Suelo desnudo, areas urbanas
- **0.2 a 0.4**: Vegetacion dispersa, pastos secos
- **0.4 a 0.6**: Vegetacion moderada
- **0.6 a 1**: Vegetacion densa y saludable

In [ ]:
# Calcular NDVI
red_float = red.astype(float)
nir_float = nir.astype(float)

# Evitar division por cero
ndvi = (nir_float - red_float) / (nir_float + red_float + 1e-10)

# Limitar valores al rango valido [-1, 1]
ndvi = ndvi.clip(-1, 1)

print(f"NDVI calculado")
print(f"Valor minimo: {float(ndvi.min()):.3f}")
print(f"Valor maximo: {float(ndvi.max()):.3f}")
print(f"Valor promedio: {float(ndvi.mean()):.3f}")

## 6. Visualizar NDVI

In [ ]:
# Crear colormap personalizado para NDVI
colors_ndvi = [
    "#d73027",  # -1 a -0.2: Rojo (agua, nubes)
    "#fc8d59",  # -0.2 a 0: Naranja
    "#fee08b",  # 0 a 0.2: Amarillo (suelo desnudo)
    "#d9ef8b",  # 0.2 a 0.4: Verde claro
    "#91cf60",  # 0.4 a 0.6: Verde medio
    "#1a9850",  # 0.6 a 0.8: Verde oscuro
    "#006837",  # 0.8 a 1: Verde muy oscuro (vegetacion densa)
]
cmap_ndvi = mcolors.LinearSegmentedColormap.from_list("ndvi", colors_ndvi, N=256)

# Visualizar
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# NDVI con colormap
im1 = axes[0].imshow(ndvi.values, cmap=cmap_ndvi, vmin=-0.2, vmax=0.8)
axes[0].set_title("NDVI - San Juan de Pasto", fontsize=14)
axes[0].axis("off")
cbar1 = plt.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04)
cbar1.set_label("NDVI")

# Histograma de NDVI
ndvi_flat = ndvi.values.flatten()
ndvi_flat = ndvi_flat[~np.isnan(ndvi_flat)]
axes[1].hist(ndvi_flat, bins=100, color="green", alpha=0.7, edgecolor="darkgreen")
axes[1].axvline(x=0.2, color="red", linestyle="--", label="Umbral vegetacion (0.2)")
axes[1].axvline(
    x=0.4, color="orange", linestyle="--", label="Vegetacion moderada (0.4)"
)
axes[1].set_xlabel("NDVI")
axes[1].set_ylabel("Frecuencia")
axes[1].set_title("Distribucion de valores NDVI", fontsize=14)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("./data/ndvi_pasto.png", dpi=150, bbox_inches="tight")
plt.show()

## 7. Clasificacion de Cobertura por NDVI

In [ ]:
# Clasificar NDVI en categorias
def clasificar_ndvi(ndvi_array):
    clasificacion = np.zeros_like(ndvi_array)
    clasificacion[ndvi_array < 0] = 1  # Agua/Nubes
    clasificacion[(ndvi_array >= 0) & (ndvi_array < 0.2)] = 2  # Suelo/Urbano
    clasificacion[(ndvi_array >= 0.2) & (ndvi_array < 0.4)] = 3  # Vegetacion dispersa
    clasificacion[(ndvi_array >= 0.4) & (ndvi_array < 0.6)] = 4  # Vegetacion moderada
    clasificacion[ndvi_array >= 0.6] = 5  # Vegetacion densa
    return clasificacion


ndvi_clasificado = clasificar_ndvi(ndvi.values)

# Contar pixeles por categoria
categorias = {
    1: "Agua/Nubes",
    2: "Suelo/Urbano",
    3: "Vegetacion dispersa",
    4: "Vegetacion moderada",
    5: "Vegetacion densa",
}

total_pixeles = ndvi_clasificado.size
print("Distribucion de cobertura en Pasto:")
print("=" * 45)
for cat_id, cat_name in categorias.items():
    count = np.sum(ndvi_clasificado == cat_id)
    porcentaje = (count / total_pixeles) * 100
    print(f"{cat_name:25s}: {porcentaje:6.2f}%")

In [ ]:
# Visualizar clasificacion
fig, ax = plt.subplots(figsize=(12, 10))

# Colormap para clasificacion
colors_class = ["#2166ac", "#d6604d", "#f4a582", "#92c5de", "#1a9850"]
cmap_class = mcolors.ListedColormap(colors_class)
bounds = [0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
norm = mcolors.BoundaryNorm(bounds, cmap_class.N)

im = ax.imshow(ndvi_clasificado, cmap=cmap_class, norm=norm)
ax.set_title("Clasificacion de Cobertura por NDVI - San Juan de Pasto", fontsize=14)
ax.axis("off")

# Leyenda
cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04, ticks=[1, 2, 3, 4, 5])
cbar.ax.set_yticklabels(
    ["Agua/Nubes", "Suelo/Urbano", "Veg. Dispersa", "Veg. Moderada", "Veg. Densa"]
)

plt.tight_layout()
plt.savefig("./data/ndvi_clasificado_pasto.png", dpi=150, bbox_inches="tight")
plt.show()

## 8. Guardar NDVI como GeoTIFF

In [ ]:
# Crear directorio si no existe
os.makedirs("./data", exist_ok=True)

# Guardar NDVI como GeoTIFF
ndvi_output = ndvi.copy()
ndvi_output.name = "ndvi"
ndvi_output.rio.to_raster("./data/ndvi_pasto.tif")

print("NDVI guardado en: ./data/ndvi_pasto.tif")

## 9. Mapa Interactivo de NDVI

In [ ]:
# Crear mapa interactivo
m = leafmap.Map(
    center=[PASTO_CENTER["lat"], PASTO_CENTER["lon"]], zoom=12, height="600px"
)

# Agregar capas base
m.add_basemap("OpenStreetMap")
m.add_basemap("Esri.WorldImagery")

# Agregar NDVI si el archivo existe
ndvi_file = "./data/ndvi_pasto.tif"
if os.path.exists(ndvi_file):
    m.add_raster(
        ndvi_file, layer_name="NDVI Pasto", colormap="RdYlGn", vmin=-0.2, vmax=0.8
    )

m.add_layer_control()
m

## 10. Analisis de Areas Especificas

In [ ]:
# Funcion para analizar NDVI en un area especifica
def analizar_ndvi_area(nir_band, red_band, bbox, nombre):
    """
    Calcula estadisticas de NDVI para un area especifica.
    """
    try:
        # Recortar al bbox
        nir_clip = nir_band.rio.clip_box(*bbox)
        red_clip = red_band.rio.clip_box(*bbox)

        # Calcular NDVI
        ndvi_local = (nir_clip.astype(float) - red_clip.astype(float)) / (
            nir_clip.astype(float) + red_clip.astype(float) + 1e-10
        )
        ndvi_local = ndvi_local.clip(-1, 1)

        # Estadisticas
        return {
            "nombre": nombre,
            "min": float(ndvi_local.min()),
            "max": float(ndvi_local.max()),
            "media": float(ndvi_local.mean()),
            "std": float(ndvi_local.std()),
            "vegetacion_pct": float((ndvi_local > 0.3).sum() / ndvi_local.size * 100),
        }
    except Exception as e:
        return {"nombre": nombre, "error": str(e)}


# Analizar cada area
print("Analisis de NDVI por areas de Pasto")
print("=" * 60)

for nombre, bbox in AREAS.items():
    stats = analizar_ndvi_area(nir, red, bbox, nombre)
    if "error" not in stats:
        print(f"\n{stats['nombre']}:")
        print(f"  NDVI Minimo: {stats['min']:.3f}")
        print(f"  NDVI Maximo: {stats['max']:.3f}")
        print(f"  NDVI Promedio: {stats['media']:.3f}")
        print(f"  Desviacion Std: {stats['std']:.3f}")
        print(f"  % con vegetacion (NDVI>0.3): {stats['vegetacion_pct']:.1f}%")

## 11. Otros Indices de Vegetacion

In [ ]:
# Calcular otros indices utiles

# EVI (Enhanced Vegetation Index) - Mejor para areas con vegetacion densa
# EVI = 2.5 * (NIR - RED) / (NIR + 6*RED - 7.5*BLUE + 1)

# Cargar banda azul si es necesario
if items:
    blue_url = best_item.assets["B02"].href
    blue = rxr.open_rasterio(blue_url).squeeze()
    blue = blue.rio.clip_box(*PASTO_BBOX)

    blue_float = blue.astype(float)

    # Calcular EVI
    evi = (
        2.5
        * (nir_float - red_float)
        / (nir_float + 6 * red_float - 7.5 * blue_float + 1 + 1e-10)
    )
    evi = evi.clip(-1, 1)

    # SAVI (Soil Adjusted Vegetation Index) - Mejor para suelos expuestos
    L = 0.5  # Factor de ajuste del suelo
    savi = ((nir_float - red_float) / (nir_float + red_float + L)) * (1 + L)
    savi = savi.clip(-1, 1)

    print(f"EVI - Promedio: {float(evi.mean()):.3f}")
    print(f"SAVI - Promedio: {float(savi.mean()):.3f}")

In [ ]:
# Comparar indices
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# NDVI
im1 = axes[0].imshow(ndvi.values, cmap="RdYlGn", vmin=-0.2, vmax=0.8)
axes[0].set_title("NDVI", fontsize=14)
axes[0].axis("off")
plt.colorbar(im1, ax=axes[0], fraction=0.046)

# EVI
im2 = axes[1].imshow(evi.values, cmap="RdYlGn", vmin=-0.2, vmax=0.8)
axes[1].set_title("EVI", fontsize=14)
axes[1].axis("off")
plt.colorbar(im2, ax=axes[1], fraction=0.046)

# SAVI
im3 = axes[2].imshow(savi.values, cmap="RdYlGn", vmin=-0.2, vmax=0.8)
axes[2].set_title("SAVI", fontsize=14)
axes[2].axis("off")
plt.colorbar(im3, ax=axes[2], fraction=0.046)

plt.suptitle("Comparacion de Indices de Vegetacion - San Juan de Pasto", fontsize=16)
plt.tight_layout()
plt.savefig("./data/indices_vegetacion_pasto.png", dpi=150, bbox_inches="tight")
plt.show()

## 12. Resumen

En este notebook hemos aprendido a:

1. **Calcular el NDVI** a partir de imagenes Sentinel-2
2. **Interpretar los valores de NDVI** para identificar tipos de cobertura
3. **Clasificar el territorio** segun la densidad de vegetacion
4. **Comparar diferentes areas** de Pasto (urbana, rural, volcanica)
5. **Calcular otros indices** como EVI y SAVI
6. **Crear visualizaciones** estaticas e interactivas

### Conclusiones para Pasto:

- El area urbana presenta valores bajos de NDVI (< 0.2)
- Las laderas del Volcan Galeras tienen vegetacion de paramo
- Las zonas rurales muestran alta actividad agricola

### Proximos pasos:

- Usar estos indices para entrenar modelos de segmentacion de uso del suelo
- Detectar cambios en la vegetacion a lo largo del tiempo